In [ ]:
file_path = "..\data\wzq-medical-cases.xlsx" # The path of original medical cases
output_file = "..\data\output.xlsx" # Define the path of the output Excel file

In [ ]:
import pandas as pd
df = pd.read_excel(file_path)
texts = df['Symptom']
print(texts[:10])

In [ ]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="zhipu_api_key") # Fill in your own APIKey
def extract(text):
    response = client.chat.completions.create(
        model="glm-3-turbo",  # Fill in the entity type to be extracted
        messages=[
            {"role": "system", "content": '''你是一位专业的医生，你的任务是将文本中的症状转换为更规范的语言表述，使其成为适合构建知识图谱的节点，将不同症状用逗号分隔，并注明对照关系，不要改动原意。
            以下为示例：
            Input:[‘头眩胀痛’, ‘咳嗽发热’, ‘鼻塞多涕’, ‘欲作呕恶’, ‘两手脉俱反关，弦劲而滑’]
            Output:头晕，头胀痛，咳嗽，发热，鼻塞，流涕，恶心欲呕，脉反管，脉弦，脉滑'''},
            {"role": "user", "content": '''将以下文本中的症状转换为规范的语言表述。
            Input: ''' + text + '''
            Output:'''}
        ],
    )
    result = response.choices[0].message
    return result

In [ ]:
with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:
    pd.DataFrame(columns=['Original symptom', 'Standardized symptom']).to_excel(writer, sheet_name='Sheet1', index=False)
    columns = ['Original symptom', 'Standardized symptom']
    n = 0
    startrow = 1
    
    for text in texts:
        n += 1
        result = extract(text).content
        fill_text = result if result else [float('NaN')]  # If there is no symptom information, fill in 'NaN'
        df = pd.DataFrame([{columns[0]: text, columns[1]: fill_text}]) # Adjust the columns in the output Excel here
        df.to_excel(writer, sheet_name='Sheet1', index=False, header=(n == 0), startrow=startrow)
        startrow += 1
        print(f"Article {n}: Symptoms of medical cases have been standardized")
        
print(f"Data saved to {output_file}")